In [1]:
pip install pylsl

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyqtgraph

Note: you may need to restart the kernel to use updated packages.


In [2]:
"""Example program to demonstrate how to read a multi-channel time-series
from LSL in a chunk-by-chunk manner (which is more efficient)."""

from pylsl import StreamInlet, resolve_stream
import numpy as np

chunk_size = 150 # 150 samples
sample_counter = 0
buffer = []

if __name__ == '__main__':
    # first resolve an EEG stream on the lab network
    print("looking for an EMG stream...")
    streams = resolve_stream('type', 'EMG_chunk')

    # create a new inlet to read from the stream
    inlet = StreamInlet(streams[0])

    while True:
        # get a new sample (you can also omit the timestamp part if you're not
        # interested in it)
        chunk, timestamps = inlet.pull_chunk()
        if timestamps:
            samples = np.array(chunk)
            sample_counter = sample_counter + samples.shape[0]
            buffer.append(samples)
            print(samples.shape)
        
        # Do something when buffer is full
        if sample_counter == chunk_size:
            # Concat list into single np array
            data = np.concatenate((buffer), axis=0)
            print(data.shape)
            print('=====================================')
            
            sample_counter = 0 # reset counter
            buffer = [] # reset buffer

looking for an EMG stream...


IndexError: list index out of range

In [3]:
#!/usr/bin/env python
"""
ReceiveAndPlot example for LSL

This example shows data from all found outlets in realtime.
It illustrates the following use cases:
- efficiently pulling data, re-using buffers
- automatically discarding older samples
- online postprocessing
"""

import numpy as np
import math
import pylsl

# Import PyQt5 modules directly, replacing the problematic import.
# Note: You might need to adjust this based on whether you're using PyQt5 or PySide2.
from PyQt5 import QtWidgets, QtCore
import pyqtgraph as pg

from pyqtgraph.Qt import QtCore, QtGui
from typing import List

# Basic parameters for the plotting window
plot_duration = 5  # how many seconds of data to show
update_interval = 60  # ms between screen updates
pull_interval = 500  # ms between each pull operation


class Inlet:
    """Base class to represent a plottable inlet"""
    def __init__(self, info: pylsl.StreamInfo):
        # create an inlet and connect it to the outlet we found earlier.
        # max_buflen is set so data older the plot_duration is discarded
        # automatically and we only pull data new enough to show it

        # Also, perform online clock synchronization so all streams are in the
        # same time domain as the local lsl_clock()
        # (see https://labstreaminglayer.readthedocs.io/projects/liblsl/ref/enums.html#_CPPv414proc_clocksync)
        # and dejitter timestamps
        self.inlet = pylsl.StreamInlet(info, max_buflen=plot_duration,
                                       processing_flags=pylsl.proc_clocksync | pylsl.proc_dejitter)
        # store the name and channel count
        self.name = info.name()
        self.channel_count = info.channel_count()

    def pull_and_plot(self, plot_time: float, plt: pg.PlotItem):
        """Pull data from the inlet and add it to the plot.
        :param plot_time: lowest timestamp that's still visible in the plot
        :param plt: the plot the data should be shown on
        """
        # We don't know what to do with a generic inlet, so we skip it.
        pass


class DataInlet(Inlet):
    """A DataInlet represents an inlet with continuous, multi-channel data that
    should be plotted as multiple lines."""
    dtypes = [[], np.float32, np.float64, None, np.int32, np.int16, np.int8, np.int64]

    def __init__(self, info: pylsl.StreamInfo, plt: pg.PlotItem):
        super().__init__(info)
        # calculate the size for our buffer, i.e. two times the displayed data
        bufsize = (2 * math.ceil(info.nominal_srate() * plot_duration), info.channel_count())
        self.buffer = np.empty(bufsize, dtype=self.dtypes[info.channel_format()])
        empty = np.array([])
        # create one curve object for each channel/line that will handle displaying the data
        self.curves = [pg.PlotCurveItem(x=empty, y=empty, autoDownsample=True) for _ in range(self.channel_count)]
        for curve in self.curves:
            plt.addItem(curve)

    def pull_and_plot(self, plot_time, plt):
        # pull the data
        _, ts = self.inlet.pull_chunk(timeout=0.0,
                                      max_samples=self.buffer.shape[0],
                                      dest_obj=self.buffer)
        # ts will be empty if no samples were pulled, a list of timestamps otherwise
        if ts:
            ts = np.asarray(ts)
            y = self.buffer[0:ts.size, :]
            this_x = None
            old_offset = 0
            new_offset = 0
            for ch_ix in range(self.channel_count):
                # we don't pull an entire screen's worth of data, so we have to
                # trim the old data and append the new data to it
                old_x, old_y = self.curves[ch_ix].getData()
                # the timestamps are identical for all channels, so we need to do
                # this calculation only once
                if ch_ix == 0:
                    # find the index of the first sample that's still visible,
                    # i.e. newer than the left border of the plot
                    old_offset = old_x.searchsorted(plot_time)
                    # same for the new data, in case we pulled more data than
                    # can be shown at once
                    new_offset = ts.searchsorted(plot_time)
                    # append new timestamps to the trimmed old timestamps
                    this_x = np.hstack((old_x[old_offset:], ts[new_offset:]))
                # append new data to the trimmed old data
                this_y = np.hstack((old_y[old_offset:], y[new_offset:, ch_ix] - ch_ix))
                # replace the old data
                self.curves[ch_ix].setData(this_x, this_y)


class MarkerInlet(Inlet):
    """A MarkerInlet shows events that happen sporadically as vertical lines"""
    def __init__(self, info: pylsl.StreamInfo):
        super().__init__(info)

    def pull_and_plot(self, plot_time, plt):
        # TODO: purge old markers
        strings, timestamps = self.inlet.pull_chunk(0)
        if timestamps:
            for string, ts in zip(strings, timestamps):
                plt.addItem(pg.InfiniteLine(ts, angle=90, movable=False, label=string[0]))


def main():
    # firstly resolve all streams that could be shown
    inlets: List[Inlet] = []
    print("looking for streams")
    streams = pylsl.resolve_streams()

    # Create the pyqtgraph window
    pw = pg.plot(title='LSL Plot')
    plt = pw.getPlotItem()
    plt.enableAutoRange(x=False, y=True)

    # iterate over found streams, creating specialized inlet objects that will
    # handle plotting the data
    for info in streams:
        if info.type() == 'Markers':
            if info.nominal_srate() != pylsl.IRREGULAR_RATE \
                    or info.channel_format() != pylsl.cf_string:
                print('Invalid marker stream ' + info.name())
            print('Adding marker inlet: ' + info.name())
            inlets.append(MarkerInlet(info))
        elif info.nominal_srate() != pylsl.IRREGULAR_RATE \
                and info.channel_format() != pylsl.cf_string:
            print('Adding data inlet: ' + info.name())
            inlets.append(DataInlet(info, plt))
        else:
            print('Don\'t know what to do with stream ' + info.name())

    def scroll():
        """Move the view so the data appears to scroll"""
        # We show data only up to a timepoint shortly before the current time
        # so new data doesn't suddenly appear in the middle of the plot
        fudge_factor = pull_interval * .002
        plot_time = pylsl.local_clock()
        pw.setXRange(plot_time - plot_duration + fudge_factor, plot_time - fudge_factor)

    def update():
        # Read data from the inlet. Use a timeout of 0.0 so we don't block GUI interaction.
        mintime = pylsl.local_clock() - plot_duration
        # call pull_and_plot for each inlet.
        # Special handling of inlet types (markers, continuous data) is done in
        # the different inlet classes.
        for inlet in inlets:
            inlet.pull_and_plot(mintime, plt)

    # create a timer that will move the view every update_interval ms
    update_timer = QtCore.QTimer()
    update_timer.timeout.connect(scroll)
    update_timer.start(update_interval)

    # create a timer that will pull and add new data occasionally
    pull_timer = QtCore.QTimer()
    pull_timer.timeout.connect(update)
    pull_timer.start(pull_interval)

    import sys

    # Start Qt event loop unless running in interactive mode or using pyside.
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtWidgets.QApplication.instance().exec_()


if __name__ == '__main__':
    main()

looking for streams


2024-03-03 23:36:40.917 (   0.466s) [          362967]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-03 23:36:40.917 (   0.466s) [          362967]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-03 23:36:40.917 (   0.466s) [          362967]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-03-03 23:36:40.917 (   0.466s) [          362967]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-03 23:36:40.917 (   0.466s) [          362967]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-03-03 23:36:40.917 (   0.466s) [          362967]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-03 23:36:40.917 (   0.466s) [          362967]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-03-03 23:36:40.917 (   0.466s) [          362967]      netinterfaces.cpp:91    I

Adding data inlet: obci_eeg1


2024-03-03 23:36:42.502 (   2.052s) [          362967]             common.cpp:66    INFO| git:v1.16.2-23-g6a85d6f9/branch:master/build:Release/compiler:AppleClang-15.0.0.15000040/link:SHARED
2024-03-04 13:54:56.692 (51498.331s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 13:55:11.900 (51513.420s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 14:13:12.981 (52594.524s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 14:13:27.997 (52609.541s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 14:13:43.009 (52624.553s) [W_obci_eeg1     ]   inlet_c

2024-03-04 15:25:20.669 (56922.414s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 15:41:00.609 (57862.390s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 15:41:15.631 (57877.404s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 15:41:30.647 (57892.418s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 15:42:29.926 (57951.697s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 15:42:44.939 (57966.710s) [W_obci_eeg1     ]   inlet_connec

2024-03-04 17:18:02.086 (63684.096s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 17:18:17.096 (63699.106s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 17:18:32.111 (63714.121s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 17:18:47.114 (63729.125s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 17:19:02.120 (63744.132s) [W_obci_eeg1     ]   inlet_connection.cpp:226    ERR| A recovery attempt encountered an unexpected error: set_option: Can't assign requested address
2024-03-04 17:19:17.128 (63759.140s) [W_obci_eeg1     ]   inlet_connec